# Learning by Necessity Interactive

Original Matlab Code: Ethan Ilzetzki


In [1]:
import numpy as np
from numpy import inf
import pandas as pd

## Put everything together

In [ ]:
R = 1.03 # real interest rate 
z = 0.3 
alph = 2/3 # typical labour share
d = 0.08 # depreciation rate
YPostWar = 0.3
YPreWar = 0.1 # 1938
YWar = [0.15, 0.325, 0.66, 1.2, 2.1, 2.4, 1.2, 0.9, 0.44] # 1939 to 1947 
PreWarLength = 5
WarLength = len(YWar)

phi = 1 # need to calibrate 
w = 0.526 
W = 0.24 * 0.25 * w
overtime = 0.5
FullTime = 0.24
delta = 0.0364
psi = 1 # need to calibrate

### Grids
T = 100
ngridLK = 50
ngridH = 10
grid = ngridLK**2 * ngridH

maxK = 20
maxL = 20
maxH = 0.35 
minH = 0.15

gridK = np.linspace(0, maxK, ngridLK)
gridL = np.linspace(0, maxL, ngridLK)
gridH = np.linspace(minH, maxH, ngridH)

### Matrices
HH_first = np.transpose(np.ones(ngridLK**2))
HH_second = np.kron(HH_first, np.transpose(gridH))
HH = np.tile(HH_second, (ngridLK**2,1)).transpose()

LLNext_first = np.kron(gridL.transpose(), np.ones(ngridH))
LLNext_second = np.kron(np.ones(ngridLK), LLNext_first)
LLNext = np.tile(LLNext_second, (ngridLK**2,1)).transpose()

KKNext_first = np.kron(gridK.transpose(), np.ones(ngridH * ngridLK))
KKNext = np.tile(KKNext_first, (ngridLK**2,1)).transpose()

LL_first = np.kron(np.ones(ngridLK), gridL)
LL = np.tile(LL_first, (grid,1))

KK_first = np.kron(gridK, np.ones(ngridLK))
KK = np.tile(KK_first, (grid,1))

Y = [[YPreWar], YWar, [YPostWar] * (T-WarLength-1)]
Y = list(np.concatenate(Y).flat) # flatten list

CostNPV = np.zeros((ngridLK**2, T+1))
Choices = np.zeros((ngridLK**2, T))
CostNow = np.zeros_like(HH)

### Cost Minimisation 
for t in range(T, 0, -1):
    step = t 
    
    UU_first = Y[t-1]/z
    UU_second = np.power((HH * LL), alph)
    UU_third = np.power(np.divide(UU_first, UU_second), 1/(1-alph))
    UU = np.divide(UU_third, KK)
    
    II = KKNext - np.multiply((1-d), KK)
    DD = LLNext - LL
    LaborCosts = W + w* HH + w*overtime*(HH-FullTime) * (HH > FullTime)
    
    CostNow_first = phi/2 * np.power(np.divide(II, KK) -d, 2) + (R-1) * KK
    CostNow_second = delta * np.divide(UU, (1-UU)) * KK
    CostNow_third = LaborCosts * (psi/2) * np.divide(np.power(DD, 2), LL)
    CostNow_fourth = LaborCosts * LL
    CostNow = CostNow_first + CostNow_second + CostNow_third + CostNow_fourth
    CostNow[UU>1] = inf
    
    CostNext_first = np.kron(CostNPV[:, t], np.ones(ngridH)) # be careful of whether it is t or t + 1
    CostNext = np.tile(CostNext_first, (ngridLK**2,1)).transpose()
    
    CostNPV[:,t-1] = np.amin((CostNow + np.divide(CostNext, R)), axis = 0) # column min 
    Choices[:, t-1] = np.argmin((CostNow + np.divide(CostNext, R)), axis = 0) 

/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in true_divide
/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in true_divide
/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: RuntimeWarning: divide by zero encountered in true_divide
/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in true_divide
/Users/chanmunfai/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in true_divide


In [24]:
Choices

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 24., 22., 22.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 27., 25., 25.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 24., 37., 37.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 29., 21., 35., 40.]])

In [25]:
pd.DataFrame(CostNPV)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.0
1,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.0
2,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.0
3,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.0
4,inf,inf,inf,inf,inf,inf,inf,inf,inf,3.059475,1.488356,0.755176,0.0
5,inf,inf,inf,inf,inf,inf,inf,inf,inf,2.637703,1.582048,0.802714,0.0
6,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,0.0
7,inf,inf,inf,inf,inf,inf,inf,inf,inf,2.736610,1.654494,0.839472,0.0
8,inf,inf,inf,inf,inf,inf,inf,inf,6.919999,2.807053,1.815062,0.931003,0.0


In [7]:
Y[t]

0.15

In [10]:
for t in range(T, 0, -1):
    print(t)

12
11
10
9
8
7
6
5
4
3
2
1


In [22]:
KK

array([[ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5.,  5.,  5., 10., 10., 10.],
       [ 0.,  0.,  0.,  5., 

In [20]:
UU

array([[       inf,        inf,        inf,        inf, 0.47407407,
        0.11851852,        inf, 0.23703704, 0.05925926],
       [       inf,        inf,        inf,        inf, 0.26666667,
        0.06666667,        inf, 0.13333333, 0.03333333],
       [       inf,        inf,        inf,        inf, 0.17066667,
        0.04266667,        inf, 0.08533333, 0.02133333],
       [       inf,        inf,        inf,        inf, 0.11851852,
        0.02962963,        inf, 0.05925926, 0.01481481],
       [       inf,        inf,        inf,        inf, 0.08707483,
        0.02176871,        inf, 0.04353741, 0.01088435],
       [       inf,        inf,        inf,        inf, 0.47407407,
        0.11851852,        inf, 0.23703704, 0.05925926],
       [       inf,        inf,        inf,        inf, 0.26666667,
        0.06666667,        inf, 0.13333333, 0.03333333],
       [       inf,        inf,        inf,        inf, 0.17066667,
        0.04266667,        inf, 0.08533333, 0.02133333],
